In [1]:
import tensorflow as tf
import numpy as np
import random

In [2]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import label_binarize
iris = load_iris()

iris_d = np.column_stack([iris.data, iris.target])

np.random.shuffle(iris_d)
#Разъединим признаки и ответы
x, y_nb = iris_d[:,:-1], iris_d[:, -1]
#Получим список возможных откликов
classes = list(set(y_nb))


In [3]:
y = label_binarize(y_nb, classes=classes)

In [4]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 15
display_step = 1

# Network Parameters
n_hidden_1 = 15 # 1st layer number of neurons
n_hidden_2 = 15 # 2nd layer number of neurons
n_input = x.shape[1]
n_classes = len(classes)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 15 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 15 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(x.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            #random_batch = np.random.random_integers(0, x.shape[0] - 1, batch_size)
            random_batch = random.sample(range(x.shape[0]), batch_size)
            batch_x, batch_y = x[random_batch, :], y[random_batch, :]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: x, Y: y}))

Epoch: 0001 cost=160.099026489
Epoch: 0002 cost=134.012319183
Epoch: 0003 cost=114.993531799
Epoch: 0004 cost=99.644949341
Epoch: 0005 cost=80.936328125
Epoch: 0006 cost=64.329825974
Epoch: 0007 cost=41.945171738
Epoch: 0008 cost=30.203640938
Epoch: 0009 cost=15.154050112
Epoch: 0010 cost=6.755579126
Epoch: 0011 cost=1.167710201
Epoch: 0012 cost=0.309132853
Epoch: 0013 cost=0.267552193
Epoch: 0014 cost=0.282455415
Epoch: 0015 cost=0.158783930
Optimization Finished!
Accuracy: 0.96
